In [1]:
from cje import http_request

request = http_request.HttpRequest()

7faa3494c2cd14b3eccb512af9f783ae


1479190078 XNQSMB 1330428213
starting to set login_data


Set-Cookie3: ALC="ac%3D2%26bt%3D1479190079%26cv%3D5.0%26et%3D1510726079%26scf%3D%26uid%3D6027133031%26vf%3D0%26vs%3D0%26vt%3D0%26es%3De9afbf1fd755b7bb450cabfb629236bf"; path="/"; domain=".login.sina.com.cn"; path_spec; expires="2017-11-15 06:07:59Z"; httponly=None; version=0
Set-Cookie3: LT=1479190079; path="/"; domain=".login.sina.com.cn"; path_spec; discard; version=0
Set-Cookie3: tgc="TGT-NjAyNzEzMzAzMQ==-1479190078-gz-86A873F0DD3746484DFED1FECEF8DAD0"; path="/"; domain=".login.sina.com.cn"; path_spec; discard; Httponly=None; version=0
Set-Cookie3: SRF=1479190079; path="/"; domain=".passport.weibo.com"; path_spec; domain_dot; expires="2026-11-13 06:07:59Z"; version=0
Set-Cookie3: SRT="D.QqHBJZ4oOGMTW!MbPGYGS4HLibSP4OYuJrsuMDHHNEYdPsWhA-upMERt4EPKRcsrAcPJ4cHNTsVtJFHgJ4oZNdYZdc9bJri8WEHwT!WfUPE4A-oBI8t7*B.vAflW-P9Rc0lR-ykjDvnJqiQVbiRVPBtS!r3JZPQVqbgVdWiMZ4siOzu4DbmKPWfNb99S3H1N!SMS!9!4myOP49HU4uNi49ndDPIJcYPSrnlMcyoi-P6OQBbJF43SOvpJcM1OFyHJEbJ5mkiODmkS4noIQEJ5mjkOmzkI4noIOHJ5mjkOmH6U!

In [2]:
uid_list = ["RoseannePark", "3977696020", "3287213664"]
html = request.get_html("http://weibo.com/" + uid_list[0])
# html = html.replace('\r', '').replace('\n', '').replace('\t', '')

In [3]:
import re
import json

scripts = re.findall("<script>FM.view\((.+?)\)</script>", html)
for script in scripts:
    script_json = json.loads(script)
    if script_json['domid'] == 'Pl_Core_T8CustomTriColumn__3':
        html_doc = script_json['html']
html_doc.replace('\r', '').replace('\n', '').replace('\t', '')

u'<div class="WB_cardwrap S_bg2" ><div class="PCD_counter"><div class="WB_innerwrap"><table class="tb_counter" cellpadding="0" cellspacing="0"><tbody><tr><td class="S_line1"><a bpfilter="page_frame"  class="t_link S_txt1" href="http://weibo.com/p/1005055609392620/follow?from=page_100505&wvr=6&mod=headfollow#place" ><strong class="W_f18">34</strong><span class="S_txt2">\u5173\u6ce8</span></a></td><td class="S_line1"><a bpfilter="page_frame"  class="t_link S_txt1" href="http://weibo.com/p/1005055609392620/follow?relate=fans&from=100505&wvr=6&mod=headfans&current=fans#place" ><strong class="W_f18">9089</strong><span class="S_txt2">\u7c89\u4e1d</span></a></td><td class="S_line1"><a bpfilter="page_frame"  class="t_link S_txt1" href="http://weibo.com/p/1005055609392620/home?from=page_100505_profile&wvr=6&mod=data#place" ><strong class="W_f18">1314</strong><span class="S_txt2">\u5fae\u535a</span></a></td></tr></tbody></table></div></div></div>'

In [64]:
from __future__ import print_function
import re
import json
from collections import defaultdict

from lxml import etree
from bs4 import BeautifulSoup

from .http_request import HttpRequest


class AutoCrawler(object):
    """
    自动爬取器，从xml rule中读取任务执行
    """

    def __init__(self, http_request):
        if http_request is None:
            self.http_request = HttpRequest()
        else:
            self.http_request = http_request

    def crawl(self, url, rule=None):
        """
        爬取操作
        :param url: 页面url
        :param rule: 页面解析rule的定义XML文档
        :type rule: string
        :return: 提取对象
        :rtype: dict
        """
        if rule is None:
            print("rule must be set")
        html = self.http_request.get_html(url)
        return self.extract(html, rule)

    def extract(self, html, rule, data):
        """
        根据rule指定的抓取规则，从页面内容中提取所需的对象
        :param html: 原始内容
        :param rule: 抓取规则
        :type rule: string
        :return: 提取对象
        :rtype: dict
        """
        doc = html
        root = etree.XML(rule)
        return self._extract(doc, root, data=data)

    def _extract(self, doc, rule, data=None):
        if not data:
            data = defaultdict()
        doc_data = self._this_extract(doc, rule)
        doc_data = list(doc_data)
        doc_data = doc_data if doc_data else [(doc, data)]
        for this_doc, this_data in doc_data:
            data = dict(data, **this_data)
            # 获取附件节点的数据合并
            attachments = [child for child in rule if child.get('nodetype') == 'attach']
            for attachment in attachments:
                attachment_doc, attachment_data = self._attachment_extract(this_doc, attachment)
                data = dict(data, **attachment_data)
            # 如果子节点全是叶子，直接返回
            leafs = [child for child in rule if child.get('nodetype') == 'leaf']
            if leafs:
                for leaf in leafs:
                    for leaf_doc, leaf_data in self._this_extract(this_doc, leaf):
                        data = dict(data, **leaf_data)
                        # print("leaf data : %s " % data)
                yield None, None, data
            # 沿主要节点向下探索
            mains = [child for child in rule if child.get('nodetype') == 'main']
            for main in mains:
                for doc, rule, data in self._extract(this_doc, main, data=data):
                    yield doc, rule, data

    def _attachment_extract(self, doc, rule):
        """
        自下而上的收集附件节点数据至顶层节点
        :param doc: 文档
        :param rule: 解析规则，是一个etree结点
        :return: 数据
        """
        data = defaultdict()
        doc_data = self._this_extract(doc, rule)
        for this_doc, this_data in doc_data:
            data = dict(data, **this_data)
            for child in rule:
                child_doc, child_data = self._attachment_extract(this_doc, child)
                data = dict(data, **child_data)
        return None, data

    def _this_extract(self, doc, root):
        data = defaultdict()
        print("==%s" % root)
        if root.tag != 'parse':
            return
        if root.attrib['type'] == 'json':
            jsonpath = root.attrib['jsonpath']
            doc_json = json.loads(doc.strip())
            # TODO 目前只有1级，TODO多级的jsonpath
            if root.get('name'):
                data = dict(data, **{root.get('name'): doc_json.get(jsonpath)})
            yield doc_json.get(jsonpath), data
        if root.attrib['type'] == 're':
            from xml.sax.saxutils import unescape
            restr = unescape(root.attrib['restr'])
            print("re : " + restr)
            for item in re.findall(restr, doc):
                # print("re result:" + str(item))
                if root.get('name'):
                    data = dict(data, **{root.get('name'): "ok"})
                if root.attrib['filter']:
                    doc_json = json.loads(item.replace('\n', '').strip())
                    key, value = root.attrib['filter'].split('=')
                    if value in doc_json.get(key):
                        print("json filter: " + root.attrib['filter'])
                        yield item.replace('\n', '').strip(), data
                    else:
                        continue
                else:
                    yield item.replace('\n', '').strip(), data
        if root.attrib['type'] == 'soup':
            print("soup")
            soup = BeautifulSoup(doc, 'lxml')
            cssselector = root.attrib['cssselector']
            # print(soup.select(cssselector))
            for sub_soup in soup.select(cssselector):
                if root.get('name'):
                    if root.get('value'):
                        data = dict(data, **{root.get('name'): sub_soup.get(root.get('value'))})
                    elif root.get('defaultvalue'):
                        data = dict(data, **{root.get('name'): root.get('defaultvalue')})
                    else:
                        data = dict(data, **{root.get('name'): sub_soup.string})
                yield sub_soup.prettify(), data

    def urls(self, url_template="http://www.baidu.com", **kwargs):
        # TODO 2 or 3 list
        # TODO 使用pop强化kwargs的限定
        """
        从url模板获取urls
        :param url_template: url模板
        :param kwargs: url定义的
        """
        for _, list_item in self._get_list_arg(kwargs):
            sub_kwargs = dict(kwargs)
            sub_kwargs['page'] = list_item
            yield url_template.format(**sub_kwargs)

    @staticmethod
    def _get_list_arg(kwargs):
        for key, value in kwargs.items():
            if type(value) is list:
                return key, value

In [65]:
from xml.sax.saxutils import escape

rule = '''
<root>
    <parse type='re' restr="%s" nodetype='main' filter='domid=Pl_Official_MyProfileFeed'>
        <parse type='json' jsonpath='html' nodetype='main'>
            <parse type='soup' cssselector='div[action-type="feed_list_item"]' nodetype='main'>
                <parse type='soup' cssselector='div:nth-of-type(1)' name="mid" value='mid'  nodetype='leaf'/>
                <parse type='soup' cssselector='div[node-type="feed_list_content"]' name="content"  nodetype='leaf'/>
                <parse type='soup' cssselector='span[node-type="forward_btn_text"] > span > em:nth-of-type(2)' name="reposts_count" nodetype='leaf'/>
                <parse type='soup' cssselector='span[node-type="comment_btn_text"] > span > em:nth-of-type(2)' name="comments_count" nodetype='leaf'/>
                <parse type='soup' cssselector='span[node-type="like_status"] > span > em:nth-of-type(2)' name="attitudes_count" nodetype='leaf'/>
                <parse type='soup' cssselector='a[node-type="feed_list_item_date"]' value='title' name="date" nodetype='leaf'/>
                <parse type='soup' cssselector='.WB_from a' value='href' name="feed_url" nodetype='leaf'/>
            </parse>
        </parse>
    </parse>
    <parse type='re' restr="%s" nodetype='attach' filter='domid=Pl_Core_T8CustomTriColumn__3'>
        <parse type='json' jsonpath='html' nodetype='attach'>
            <parse type='soup' cssselector='strong:nth-of-type(1)' name="friends_count" nodetype='attach'/>
            <parse type='soup' cssselector='strong:nth-of-type(2)' name="followers_count" nodetype='attach'/>
            <parse type='soup' cssselector='strong:nth-of-type(3)' name="statuses_count" nodetype='attach'/>
        </parse>
    </parse>
    <parse type='re' restr="%s" nodetype='attach' filter='domid=Pl_Official_Headerv6__1'>
        <parse type='json' jsonpath='html' nodetype='attach'>
            <parse type='soup' cssselector='img:nth-of-type(1)' name="profile_image_url" value='src'  nodetype='leaf'/>
            <parse type='soup' cssselector='a[action-type="webim.conversation"]' value='action-data' name="uid_nick" nodetype='attach'/>
            <parse type='soup' cssselector='i.icon_pf_male' name="male" nodetype='attach'/>
            <parse type='soup' cssselector='i.icon_pf_female' name="female" defaultvalue='1' nodetype='attach'/>
        </parse>
    </parse>
</root>
''' % (escape("<script>FM.view\((.+?)\)</script>"), escape("<script>FM.view\((.+?)\)</script>"), escape("<script>FM.view\((.+?)\)</script>"))
rule = rule.replace("\n", "")
doc_rule_data = AutoCrawler(request).extract(html, rule, {"account": 1111111})
items = [data for _, _, data in doc_rule_data]

==<Element root at 0x6e41748>
==<Element parse at 0x6e41388>
re : <script>FM.view\((.+?)\)</script>
json filter: domid=Pl_Core_T8CustomTriColumn__3
==<Element parse at 0x6eed648>
==<Element parse at 0x6eed3c8>
soup
==<Element parse at 0x7c95308>
soup
==<Element parse at 0x6e41c08>
soup
==<Element parse at 0x6e41988>
re : <script>FM.view\((.+?)\)</script>
json filter: domid=Pl_Official_Headerv6__1
==<Element parse at 0x6eed648>
==<Element parse at 0x7c95548>
soup
==<Element parse at 0x6eed788>
soup
==<Element parse at 0x8431c08>
soup
==<Element parse at 0x63f9dc8>
soup
==<Element parse at 0x6eed648>
re : <script>FM.view\((.+?)\)</script>
json filter: domid=Pl_Official_MyProfileFeed
==<Element parse at 0x63f8708>
==<Element parse at 0x7e13608>
soup
==<Element parse at 0x8835ec8>
soup
==<Element parse at 0x8835488>
soup
==<Element parse at 0x88358c8>
soup
==<Element parse at 0x8835d48>
soup
==<Element parse at 0x8835608>
soup
==<Element parse at 0x8839a48>
soup
==<Element parse at 0x88399

==<Element parse at 0x8835608>
soup
==<Element parse at 0x8839a48>
soup
==<Element parse at 0x8839948>
soup
==<Element parse at 0x8835ec8>
soup
==<Element parse at 0x8835488>
soup
==<Element parse at 0x88358c8>
soup
==<Element parse at 0x8835d48>
soup
==<Element parse at 0x8835608>
soup
==<Element parse at 0x8839a48>
soup
==<Element parse at 0x8839948>
soup
==<Element parse at 0x8835ec8>
soup
==<Element parse at 0x8835488>
soup
==<Element parse at 0x88358c8>
soup
==<Element parse at 0x8835d48>
soup
==<Element parse at 0x8835608>
soup
==<Element parse at 0x8839a48>
soup
==<Element parse at 0x8839948>
soup
==<Element parse at 0x8835ec8>
soup
==<Element parse at 0x8835488>
soup
==<Element parse at 0x88358c8>
soup
==<Element parse at 0x8835d48>
soup
==<Element parse at 0x8835608>
soup
==<Element parse at 0x8839a48>
soup
==<Element parse at 0x8839948>
soup
==<Element parse at 0x8835ec8>
soup
==<Element parse at 0x8835488>
soup
==<Element parse at 0x88358c8>
soup
==<Element parse at 0x8835d4

==<Element parse at 0x8835488>
soup
==<Element parse at 0x88358c8>
soup
==<Element parse at 0x8835d48>
soup
==<Element parse at 0x8835608>
soup
==<Element parse at 0x8839a48>
soup
==<Element parse at 0x8839948>
soup
==<Element parse at 0x8835ec8>
soup
==<Element parse at 0x8835488>
soup
==<Element parse at 0x88358c8>
soup
==<Element parse at 0x8835d48>
soup
==<Element parse at 0x8835608>
soup
==<Element parse at 0x8839a48>
soup
==<Element parse at 0x8839948>
soup
==<Element parse at 0x8835ec8>
soup
==<Element parse at 0x8835488>
soup
==<Element parse at 0x88358c8>
soup
==<Element parse at 0x8835d48>
soup
==<Element parse at 0x8835608>
soup
==<Element parse at 0x8839a48>
soup
==<Element parse at 0x8839948>
soup
==<Element parse at 0x8835ec8>
soup
==<Element parse at 0x8835488>
soup
==<Element parse at 0x88358c8>
soup
==<Element parse at 0x8835d48>
soup
==<Element parse at 0x8835608>
soup
==<Element parse at 0x8839a48>
soup
==<Element parse at 0x8839948>
soup
==<Element parse at 0x8835ec

In [7]:
import math

func = eval('math.sqrt')
print(func(4))

2.0


In [7]:
import math

func = eval('math.sqrt')
print(func(4))

2.0


In [62]:
import ConfigParser
import json
from pymongo.errors import BulkWriteError

class MongoSaver(object):
    """
    MongoDB专用存储器
    """

    def __init__(self):
        super(MongoSaver, self).__init__()
        from pymongo import MongoClient
        config = ConfigParser.ConfigParser()
        config.read("F:\gitlab\mola\house\cje\data_saver.conf")
        print(config.__dict__)
        print(config.items("mongo"))
        mongo_host = config.get("mongo", "host")
        mongo_dbname = config.get("mongo", "dbname")

        self.client = MongoClient(mongo_host)
        self.db = self.client.get_database(mongo_dbname)

    def save(self, table='test', items=None):
        """
        向集合存储批量数据
        :param table: 对应mongodb的collection
        :param items: 待插入的数据
        """
        if items is None:
            items = []
        # if items and items[0].get('uid'):
        #     items = [dict(item, **{"_id": item['uid']}) for item in items]
        try:
            self.db[table].insert_many(items)
        except BulkWriteError as bwe:
            print(bwe.details)
            # you can also take this component and do more analysis
            write_errors = bwe.details['writeErrors']
            print(write_errors)
            pass



data_saver = MongoSaver()
data_saver.save(items=done_items)
# for _, _, data in doc_rule_data:
#     print("---- %s " % data)

{'_optcre': <_sre.SRE_Pattern object at 0x0000000003A2D7D8>, '_defaults': OrderedDict(), '_sections': OrderedDict([('weibo', OrderedDict([('__name__', 'weibo'), ('username', 'changshanzhendingzhaozilong@gmail.com'), ('password', '7faa3494c2cd14b3eccb512af9f783ae')])), ('qq', OrderedDict([('__name__', 'qq'), ('username', 'changshanzhendingzhaozilong@gmail.com'), ('password', '3fd722a014d3ef7f774dca3e0f545860')])), ('mongo', OrderedDict([('__name__', 'mongo'), ('host', 'mongodb://10.40.100.16:27017/'), ('dbname', 'test')])), ('hbase', OrderedDict([('__name__', 'hbase'), ('host', '10.40.100.16'), ('table', 'test')]))]), '_dict': <class 'collections.OrderedDict'>}
[('host', 'mongodb://10.40.100.16:27017/'), ('dbname', 'test')]


In [87]:
def del_mongodb_id(record):
    if type(record) == dict and '_id' in record:
        del record['_id']
    return record


def del_field(record, field_name):
    if type(record) == dict and field_name in record:
        del record[field_name]
    return record


def trans_and_add(item, func):
    trans = func(item)
    item = dict(item, **trans)  # 合并
    return item


def trans_to_json_array(meta, tuple_list):
    items = []
    for tuple_data in tuple_list:
        item = trans_to_json(meta, tuple_data)
        items.append(item)
    return items


def trans_to_json(meta, tuple_data):
    item = {}
    for key in meta:
        index = meta.index(key)
        value = tuple_data[index]
        item[key] = value
    return item


def uid_nick(item):
    uid_kv, nick_kv = item['uid_nick'].split("&")
    _, uid = uid_kv.split("=")
    _, nick = nick_kv.split("=")
    return {"uid": uid, "nick": nick}


def trans_field(item, field_name, func):
    field_value = item.get(field_name)
    item[field_name] = func(field_value)
    return item


def add_math_field(item, field_name, func):
    field_value = float(item.get(field_name))
    math_field_name = "%s_%s" % (field_name, func.__name__)
    item[field_name] = field_value
    item[math_field_name] = func(field_value)
    return item


def add_custom_field(item, field_name, func):
    field_value = item.get(field_name)
    custom_field_name = "%s_%s" % (field_name, func.__name__)
    item[custom_field_name] = func(field_value)
    return item


def ln(x):
    return math.log(x)


def int_or_zero(src):
    src = src.replace("\n", "").strip()
    try:
        dist = int(src)
    except:
        dist = 0
    return dist

op_rule = '''
<root>
    <trans ops='add_uid_nick|del_uid_nick|type_followers_count_int|math_friends_count_sqrt|math_statuses_count_log|custom__followers_count__ln
    |custom__statuses_count__ln|custom__comments_count__int_or_zero|del_comments_count|custom__reposts_count__int_or_zero|del_reposts_count' />
</root>
'''

root = etree.XML(op_rule)
ops = [op.strip() for op in child.get('ops').split('|') for child in root if child.tag == 'trans']
# ops = ['do_uid_nick']
# ops = ['add_uid_nick', 'del_uid_nick', 'type_followers_count_int', 
#        'math_friends_count_sqrt', 'math_statuses_count_log',
#        'custom__followers_count__ln', 'custom__statuses_count__ln', 
#        'custom__comments_count__int_or_zero', 'del_comments_count',
#        'custom__reposts_count__int_or_zero', 'del_reposts_count']
done_items = [item for item in items]


def do_ops_to_item(item, ops):
    """
    集中式数据变换：依次进行
    :param item: 待变换的item
    :param ops: 操作集合
    :return: 变换后的item
    """
    done_item = item
    for op in ops:
        # print(op)
        if op.startswith('add_'):
            func = eval(op.replace("add_", ''))
            done_item = trans_and_add(done_item, func)
        if op.startswith('del_'):
            field_name = op.replace("del_", '')
            done_item = del_field(done_item, field_name)
        if op.startswith('do_'):
            func = eval(op.replace("do_", ''))
            done_item = func(done_item)
        if op.startswith('type_'):
            field_name = '_'.join(op.split('_')[1:-1])
            func = eval(op.split('_')[-1])
            done_item = trans_field(done_item, field_name, func)
        if op.startswith('math_'):
            field_name = '_'.join(op.split('_')[1:-1])
            func = eval("math." + op.split('_')[-1])
            done_item = add_math_field(done_item, field_name, func)
        if op.startswith('custom__'):
            field_name = op.split('__')[1]
            func = eval(op.split('__')[-1])
            done_item = add_custom_field(done_item, field_name, func)
    return done_item

done_items = [do_ops_to_item(item, ops) for item in done_items]

# for op in ops:
#     if op.startswith('add_'):
#         func = eval(op.replace("add_", ''))
#         done_items = [trans_and_add(item, uid_nick) for item in done_items]
#     if op.startswith('del_'):
#         field_name = op.replace("del_", '')
#         done_items = [del_field(item, field_name) for item in done_items]
#     if op.startswith('do_'):
#         func = eval(op.replace("do_", ''))
#         done_items = [func(item) for item in done_items]
#     if op.startswith('type_'):
#         field_name = '_'.join(op.split('_')[1:-1])
#         func = eval(op.split('_')[-1])
#         done_items = [trans_field(item, field_name, func) for item in done_items]
#     if op.startswith('math_'):
#         field_name = '_'.join(op.split('_')[1:-1])
#         func = eval("math." + op.split('_')[-1])
#         done_items = [add_math_field(item, field_name, func) for item in done_items]
print(len(done_items))
done_items

15


[{'account': 1111111,
  'comments_count_int_or_zero': 21,
  'content': None,
  'date': '2016-11-12 21:10',
  'feed_url': 'http://weibo.com/p/1008080d055beb31fd94ce0399a1981d82afc5?k=BLACKPINK&from=feed_card',
  'female': '1',
  'followers_count': 9089,
  'followers_count_ln': 9.11482017011877,
  'friends_count': 34.0,
  'friends_count_sqrt': 5.830951894845301,
  'mid': '4041104175715421',
  'nick': u'ROSE\u5427\u5b98\u535a',
  'profile_image_url': 'http://tva4.sinaimg.cn/crop.0.0.512.512.180/0067Crgwjw8f91xwrgvwoj30e80e8jrh.jpg',
  'reposts_count_int_or_zero': 47,
  'statuses_count': 1314.0,
  'statuses_count_ln': 7.1808311990445555,
  'statuses_count_log': 7.1808311990445555,
  'uid': u'5609392620'},
 {'account': 1111111,
  'comments_count_int_or_zero': 8,
  'content': None,
  'date': '2016-11-12 14:39',
  'feed_url': 'http://vip.weibo.com/prividesc?priv=1006&from=feed',
  'female': '1',
  'followers_count': 9089,
  'followers_count_ln': 9.11482017011877,
  'friends_count': 34.0,
  'f

In [7]:
import math

func = eval('math.sqrt')
print(func(4))

2.0


In [84]:
from __future__ import print_function

op_rule = '''
<root>
    <trans ops='add_uid_nick|del_uid_nick|type_followers_count_int|math_friends_count_sqrt|math_statuses_count_log|custom__followers_count__ln
    |custom__statuses_count__ln|custom__comments_count__int_or_zero|del_comments_count|custom__reposts_count__int_or_zero|del_reposts_count' />
</root>
'''

root = etree.XML(op_rule)
targets = [op.strip() for op in child.get('ops').split('|') for child in root]
print(targets)
# ops = ['do_uid_nick']
ops = ['add_uid_nick', 'del_uid_nick', 'type_followers_count_int', 'math_friends_count_sqrt', 'math_statuses_count_log',
       'custom__followers_count__ln', 'custom__statuses_count__ln', 'custom__comments_count__int_or_zero', 'del_comments_count',
       'custom__reposts_count__int_or_zero', 'del_reposts_count']
print(" | ".join(ops))
print('\n|'.strip())

['add_uid_nick', 'del_uid_nick', 'type_followers_count_int', 'math_friends_count_sqrt', 'math_statuses_count_log', 'custom__followers_count__ln', 'custom__statuses_count__ln', 'custom__comments_count__int_or_zero', 'del_comments_count', 'custom__reposts_count__int_or_zero', 'del_reposts_count']
add_uid_nick | del_uid_nick | type_followers_count_int | math_friends_count_sqrt | math_statuses_count_log | custom__followers_count__ln | custom__statuses_count__ln | custom__comments_count__int_or_zero | del_comments_count | custom__reposts_count__int_or_zero | del_reposts_count
|


In [38]:
print("Pl_Official_MyProfileFeed" in "Pl_Official_MyProfileFeed_22")

True


In [9]:
def a():
    yield 1


def b():
    for i in range(10):
        for sa in a():
            yield sa

for sb in b():
    print(sb)

1
1
1
1
1
1
1
1
1
1
